In [1]:
import codecs
import json
import logging
import os
import sys
import operator
from collections import defaultdict
from tokenizer import twokenize
from twitter.Tweet import Tweet
from util import locations, ngrams
from processing import locationsInTweets
import stats_enriched_tweets

In [2]:
print os.getcwd()

/Users/cris/Documents/workspace/TwitterCrisis


In [3]:
# read enriched tweet
def tweetIter(inputfile):
    for line in codecs.open(inputfile, "r", "utf-8"):
        try:
            tweet = json.loads(line)
        except:
            print "Couldn't parse tweet: ", line[:200]
        yield tweet

In [4]:
def hasBothLocationAndMention(tweet):
    return (tweet["final_location"] or tweet["final_location_c"]) and (tweet["text_location_mentions"] or tweet["text_location_mentions_c"])

In [ ]:
# Read file with simplified schema
'''
sentiment_tweet: STRING 'pro_ref' : pro refugees, 'against_ref' : against refugees 
                    (the tweet contains polarized hashtags)
sentiment_user: BOOLEAN  1 : pro refugees, 0 : against refugees 
                    (the user is polarized by analyzing the tweets he/she wrote so far)
'''


#inputFilename = "/Users/cris/all-brexit.json"
inputFilename = "/Users/cris/all-brexit-with-sent_pre.json"
tweetsAsDict = tweetIter(inputFilename)

# user sentiment count
countUserPro = 0 
countUserAgainst = 0

# tweet sentiment count
countTweetPro = 0
countTweetAgainst = 0

# user location - from final
userLocDict = defaultdict(int)
userLocCountryDict = defaultdict(int)
countFinalLoc = 0
countFinalLocCountry = 0

# location mention
locationMention = defaultdict(int)
locationCountryMention = defaultdict(int)
countLocMention = 0
countLocMention_c = 0

# both location and location mention
countBoth = 0
countBoth2 = 0


i = 0
for tweet in tweetsAsDict:
    i+=1
    
    if "sentiment_user" in tweet:
        if tweet["sentiment_user"] == 1:
            countUserPro += 1
        else:
            countUserAgainst += 1
            
            
    if "sentiment_tweet" in tweet:
        if tweet["sentiment_tweet"] == 'pro_ref':
            countTweetPro += 1
        else:
            countTweetAgainst += 1
            
    if tweet["final_location"]:
        countFinalLoc += 1
        userLocDict[tweet["final_location"]] +=1
            
    if tweet["final_location_c"]:
        countFinalLocCountry += 1
        userLocCountryDict[tweet["final_location_c"]] +=1
    
    if tweet["text_location_mentions"]:
        if len(tweet["text_location_mentions"]) > 1:
            for loc in tweet["text_location_mentions"]:
                countLocMention += 1
                locationMention[loc] +=1
        else:
            countLocMention += 1
            locationMention[tweet["text_location_mentions"][0]] +=1
            
    if tweet["text_location_mentions_c"]:
        if len(tweet["text_location_mentions_c"]) > 1:
            for loc_c in tweet["text_location_mentions_c"]:
                countLocMention_c += 1
                locationCountryMention[loc_c] +=1
        else:
            countLocMention_c += 1
            locationCountryMention[tweet["text_location_mentions_c"][0]] +=1
            
    if hasBothLocationAndMention(tweet):
        countBoth += 1
        
    if stats_enriched_tweets.hasUserLocationAndMentionLocation(tweet):
        countBoth2 +=1
    
#     if i%10000 ==0 :
#         break
        
print "Tweets with User pro: ", countUserPro, "Tweets with User against: ", countUserAgainst
print "Tweets pro: ", countTweetPro, "Tweets against: ", countTweetAgainst
print "Final loc: ", countFinalLoc, "Final loc country: ", countFinalLocCountry
print "Loc mention: ", countLocMention, "Loc mention country: ", countLocMention_c
print "With Loc and loc mention: ", countBoth, countBoth2

print i

In [ ]:
b = sorted(locationMention.items(), key=operator.itemgetter(1), reverse=True)
print "Cities: ", len(b), b

In [ ]:
a = sorted(locationCountryMention.items(), key=operator.itemgetter(1), reverse=True)

In [ ]:
print "Countries: ", len(a), a